使用textRNN进行文本分类，还是使用flyai的框架

定义参数

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("-e", "--EPOCHS", default=50, type=int, help="train epochs")
parser.add_argument("-b", "--BATCH", default=64, type=int, help="batch size")
args = parser.parse_args()

'''
flyai库中的提供的数据处理方法
传入整个数据训练多少轮，每批次批大小
'''
dataset = Dataset(epochs=args.EPOCHS, batch=args.BATCH)
model = Model(dataset)

# 超参
vocab_size = 20655      # 总词汇量
embedding_dim = 100      # 嵌入层大小
hidden_dim = 100        # Dense层大小
max_seq_len = 34        # 最大句长
num_filters = 256       # 卷积核数目
kernel_size = 5         # 卷积核尺寸
learning_rate = 1e-3    # 学习率
numclass = 3            # 类别数
num_layers = 2


# 传值空间
input_x = tf.placeholder(tf.int32, shape=[None, max_seq_len], name='input_x')
input_y = tf.placeholder(tf.int32, shape=[None], name='input_y')
keep_prob = tf.placeholder(tf.float32, name='keep_prob')


定义网络

In [ ]:

with tf.name_scope("textrnn"):
    # CNN layer
    x_input_embedded = tf.nn.embedding_lookup(input_embedding, input_x)
    cell = tf.nn.rnn_cell.LSTMCell(hidden_dim)
    cell = tf.nn.rnn_cell.DropoutWrapper(cell=cell, output_keep_prob=keep_prob)
    cells = [cell for _ in range(num_layers)]
    Cell = tf.nn.rnn_cell.MultiRNNCell(cells, state_is_tuple=True)

    output, _ = tf.nn.dynamic_rnn(cell=Cell, inputs=x_input_embedded,
                                  dtype=tf.float32)
    output = tf.reduce_sum(output, axis=1)
    dropout = tf.nn.dropout(output, keep_prob=keep_prob)
    W = tf.Variable(tf.truncated_normal([hidden_dim, numclass], stddev=0.1))
    b = tf.Variable(tf.constant(0.1, shape=[numclass]))
    logits = tf.matmul(dropout, W) + b
    # conv = tf.layers.conv1d(x_input_embedded, num_filters, kernel_size, name='conv')
    # # global max pooling layer
    # pooling = tf.reduce_max(conv, reduction_indices=[1])


with tf.name_scope("score"):
    # 全连接层，后面接dropout以及relu激活



    y_pred_cls = tf.argmax(tf.nn.softmax(logits), 1, name='y_pred')  # 预测类别

with tf.name_scope("optimize"):
    # 将label进行onehot转化
    one_hot_labels = tf.one_hot(input_y, depth=numclass, dtype=tf.float32)
    # 损失函数，交叉熵
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=one_hot_labels)
    loss = tf.reduce_mean(cross_entropy)
    # 优化器
    train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

训练

In [ ]:
with tf.name_scope("accuracy"):
    # 准确率
    correct_pred = tf.equal(tf.argmax(one_hot_labels, 1), y_pred_cls)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='acc')

with tf.name_scope("summary"):
    tf.summary.scalar("loss", loss)
    tf.summary.scalar("accuracy", accuracy)
    merged_summary = tf.summary.merge_all()

best_score = 0
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    train_writer = tf.summary.FileWriter(LOG_PATH, sess.graph)

    # dataset.get_step() 获取数据的总迭代次数
    for step in range(dataset.get_step()):
        x_train, y_train = dataset.next_train_batch()
        # x_val, y_val = dataset.next_validation_batch()

        fetches = [loss, accuracy, train_op]
        feed_dict = {input_x: x_train, input_y: y_train, keep_prob: 0.9}
        loss_, accuracy_, _ = sess.run(fetches, feed_dict=feed_dict)

        summary = sess.run(merged_summary, feed_dict=feed_dict)
        train_writer.add_summary(summary, step)

        cur_step = str(step + 1) + "/" + str(dataset.get_step())
        print('The Current step per total: {} | The Current loss: {} | The Current ACC: {}'.
              format(cur_step, loss_, accuracy_))
        if step % 100 == 0:
            model.save_model(sess, MODEL_PATH, overwrite=True)

可以看出结果不错

In [ ]:
The Current step per total: 90/100 | The Current loss: 2.2351731132630448e-08 | The Current ACC: 1.0
The Current step per total: 91/100 | The Current loss: 7.778337021591142e-06 | The Current ACC: 1.0
The Current step per total: 92/100 | The Current loss: 7.450578820566989e-09 | The Current ACC: 1.0
The Current step per total: 93/100 | The Current loss: 1.8626450382086546e-09 | The Current ACC: 1.0
The Current step per total: 94/100 | The Current loss: 5.587934559514451e-09 | The Current ACC: 1.0
The Current step per total: 95/100 | The Current loss: 1.1175868230850483e-08 | The Current ACC: 1.0
The Current step per total: 96/100 | The Current loss: 1.8626450382086546e-09 | The Current ACC: 1.0
The Current step per total: 97/100 | The Current loss: 7.4505797087454084e-09 | The Current ACC: 1.0
The Current step per total: 98/100 | The Current loss: 3.7252898543727042e-09 | The Current ACC: 1.0
The Current step per total: 99/100 | The Current loss: 3.0174564358276257e-07 | The Current ACC: 1.0
The Current step per total: 100/100 | The Current loss: 5.587935003603661e-09 | The Current ACC: 1.0


接下来使用RCNN进行文本分类

定义参数

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("-e", "--EPOCHS", default=10, type=int, help="train epochs")
parser.add_argument("-b", "--BATCH", default=64, type=int, help="batch size")
args = parser.parse_args()

'''
flyai库中的提供的数据处理方法
传入整个数据训练多少轮，每批次批大小
'''
dataset = Dataset(epochs=args.EPOCHS, batch=args.BATCH)
model = Model(dataset)

# 超参
vocab_size = 20655      # 总词汇量
embedding_dim = 300      # 嵌入层大小
hidden_dim = 512        # Dense层大小
max_seq_len = 34        # 最大句长
num_filters = 256       # 卷积核数目
kernel_size = 5         # 卷积核尺寸
learning_rate = 1e-3    # 学习率
numclass = 3            # 类别数
text_hidden_size = 512

# 传值空间
input_x = tf.placeholder(tf.int32, shape=[None, max_seq_len], name='input_x')
input_y = tf.placeholder(tf.int32, shape=[None], name='input_y')
keep_prob = tf.placeholder(tf.float32, name='keep_prob')


# define embedding layer
with tf.variable_scope('embedding'):
    # 标准正态分布初始化
    input_embedding = tf.Variable(
        tf.truncated_normal(shape=[vocab_size, embedding_dim], stddev=0.1), name='encoder_embedding')


定义网络

In [ ]:
with tf.name_scope("rcnn"):
    # CNN layer
    x_input_embedded = tf.nn.embedding_lookup(input_embedding, input_x)
    # conv = tf.layers.conv1d(x_input_embedded, num_filters, kernel_size, name='conv')
    # # global max pooling layer
    # pooling = tf.reduce_max(conv, reduction_indices=[1])
    fw_cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_dim)
    fw_cell = tf.nn.rnn_cell.DropoutWrapper(fw_cell, output_keep_prob=keep_prob)
    bw_cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_dim)
    bw_cell = tf.nn.rnn_cell.DropoutWrapper(bw_cell, output_keep_prob=keep_prob)
    (output_fw, output_bw), states = tf.nn.bidirectional_dynamic_rnn(
        cell_fw=fw_cell, cell_bw=bw_cell, inputs=x_input_embedded, dtype=tf.float32
    )
    shape = [tf.shape(output_fw)[0], 1, tf.shape(output_fw)[2]]
    c_left = tf.concat([tf.zeros(shape), output_fw[:, :-1]], axis=1)
    c_right = tf.concat([output_bw[:, 1:], tf.zeros(shape)], axis=1)
    x = tf.concat([c_left, x_input_embedded, c_right], axis=2)
    embedding_size = 2 * hidden_dim + embedding_dim
    W2 = tf.Variable(tf.random_uniform([embedding_size, text_hidden_size], -1.0, 1.0))
    b2 = tf.Variable(tf.constant(0.1, shape=[text_hidden_size]))
    y2 = tf.tanh(tf.einsum('aij, jk->aik', x, W2) + b2)
    y3 = tf.reduce_max(y2, axis=1)
    W4 = tf.get_variable("W4", shape=[text_hidden_size, numclass],
                         initializer=tf.contrib.layers.xavier_initializer())
    b4 = tf.Variable(tf.constant(0.1, shape=[numclass]))
with tf.name_scope("score"):
    # # 全连接层，后面接dropout以及relu激活
    # fc = tf.layers.dense(pooling, hidden_dim, name='fc1')
    # fc = tf.contrib.layers.dropout(fc, keep_prob)
    # fc = tf.nn.relu(fc)
    #
    # # 分类器
    # logits = tf.layers.dense(fc, numclass, name='fc2')
    logits = tf.nn.xw_plus_b(y3, W4, b4)
    y_pred_cls = tf.argmax(tf.nn.softmax(logits), 1, name='y_pred')  # 预测类别

with tf.name_scope("optimize"):
    # 将label进行onehot转化
    one_hot_labels = tf.one_hot(input_y, depth=numclass, dtype=tf.float32)
    # 损失函数，交叉熵
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=one_hot_labels)
    loss = tf.reduce_mean(cross_entropy)
    # 优化器
    train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)


训练

In [ ]:
with tf.name_scope("accuracy"):
    # 准确率
    correct_pred = tf.equal(tf.argmax(one_hot_labels, 1), y_pred_cls)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='acc')

with tf.name_scope("summary"):
    tf.summary.scalar("loss", loss)
    tf.summary.scalar("accuracy", accuracy)
    merged_summary = tf.summary.merge_all()

best_score = 0
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    train_writer = tf.summary.FileWriter(LOG_PATH, sess.graph)

    # dataset.get_step() 获取数据的总迭代次数
    for step in range(dataset.get_step()):
        x_train, y_train = dataset.next_train_batch()
        # x_val, y_val = dataset.next_validation_batch()

        fetches = [loss, accuracy, train_op]
        feed_dict = {input_x: x_train, input_y: y_train, keep_prob: 0.9}
        loss_, accuracy_, _ = sess.run(fetches, feed_dict=feed_dict)

        summary = sess.run(merged_summary, feed_dict=feed_dict)
        train_writer.add_summary(summary, step)

        cur_step = str(step + 1) + "/" + str(dataset.get_step())
        print('The Current step per total: {} | The Current loss: {} | The Current ACC: {}'.
              format(cur_step, loss_, accuracy_))
        if step % 100 == 0:
            model.save_model(sess, MODEL_PATH, overwrite=True)

可以看出结果不错

In [ ]:
The Current step per total: 6/20 | The Current loss: 0.5880126953125 | The Current ACC: 0.78125
The Current step per total: 7/20 | The Current loss: 0.5793024897575378 | The Current ACC: 0.78125
The Current step per total: 8/20 | The Current loss: 0.5130840539932251 | The Current ACC: 0.78125
The Current step per total: 9/20 | The Current loss: 0.4589272141456604 | The Current ACC: 0.78125
The Current step per total: 10/20 | The Current loss: 0.40918341279029846 | The Current ACC: 0.78125
The Current step per total: 11/20 | The Current loss: 0.37503427267074585 | The Current ACC: 0.78125
The Current step per total: 12/20 | The Current loss: 0.33043915033340454 | The Current ACC: 0.796875
The Current step per total: 13/20 | The Current loss: 0.28246819972991943 | The Current ACC: 0.890625
The Current step per total: 14/20 | The Current loss: 0.2591536045074463 | The Current ACC: 0.921875
The Current step per total: 15/20 | The Current loss: 0.2163408398628235 | The Current ACC: 0.90625
The Current step per total: 16/20 | The Current loss: 0.1775176227092743 | The Current ACC: 0.96875
The Current step per total: 17/20 | The Current loss: 0.14954215288162231 | The Current ACC: 0.984375
The Current step per total: 18/20 | The Current loss: 0.12818661332130432 | The Current ACC: 1.0
The Current step per total: 19/20 | The Current loss: 0.1094832792878151 | The Current ACC: 1.0
The Current step per total: 20/20 | The Current loss: 0.08871211111545563 | The Current ACC: 1.0